# MLB Project for data from the 2022-2023 season

### Dataset available from https://drive.google.com/drive/folders/1G1WHb3nBpv3xXQY4V6pLo8qI9lHq9X0_?usp=sharing


In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [12]:
df = pd.read_csv(r'2023-mlb-data.csv')
df.shape

(736626, 95)

In [3]:
# Adding more measures to data
# Total Score
df['total_score'] = df['home_score'] + df['away_score']

# Is_strike flag
df['is_strike'] = 0
strike_events = ['called_strike', 'swinging_strike', 'foul', 'swinging_strike_blocked', 'foul_tip', 'foul_bunt', 'missed_bunt', 'bunt_foul_tip']
df.loc[df['description'].isin(strike_events), 'is_strike'] = 1



In [4]:
_df = df.loc [:, ['pitcher_name', 'pitch_name', 'is_strike']].groupby(['pitcher_name', 'pitch_name'], as_index=False).count().rename(columns={'is_strike':'n_pitches'})

df2 = df.merge(_df, on=['pitcher_name', 'pitch_name'])

df2 = df2.loc[df2['n_pitches']>=100]

df2['runs_scored'] = df2['post_bat_score'] - df2['bat_score']

In [5]:
df2['runs_scored'].describe()

count    672545.000000
mean          0.028695
std           0.209070
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           4.000000
Name: runs_scored, dtype: float64

In [6]:
# Create Era dataframe/dataset
era = df2.loc[:, ['pitcher_name', 'pitcher', 'runs_scored']].groupby(['pitcher_name', 'pitcher'], as_index=False).sum()

# create innings data for era df
_df = df2.loc[:, ['pitcher', 'game_pk', 'inning']].drop_duplicates()
_df = _df['pitcher'].value_counts().to_frame().reset_index().rename(columns={'count':'n_innings'})

# merge temp df to era df
era = era.merge(_df, on=['pitcher'])

# create era metric
era['era'] = (era['runs_scored']/era['n_innings'])*9

era.head()

,pitcher_name,pitcher,runs_scored,n_innings,era
0,"Abbott, Cory",676265,25,52,4.326923
1,"Abreu, Albert",656061,12,51,2.117647
2,"Abreu, Bryan",650556,26,81,2.888889
3,"Acevedo, Domingo",642758,32,82,3.512195
4,"Adam, Jason",592094,14,77,1.636364


stikeouts per 9. Calculate stikeouts over the cours of a game 